In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
os.environ["PINECONE_API_KEY"] = ""
os.environ['OPENAI_API_KEY'] = ''
from pinecone import Pinecone, ServerlessSpec

/opt/miniconda3/envs/rag/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "rag"

pc.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [3]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Sarah Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Johnson's lectures are engaging and well-structured. She's always willing to help during office hours."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant instructor! Prof. Chen makes complex concepts easy to understand and provides great real-world examples.'},
 {'professor': 'Dr. Emily Rodriguez',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Decent course content, but Dr. Rodriguez could improve on time management during lectures.'},
 {'professor': 'Prof. David Thompson',
  'subject': 'History',
  'stars': 4,
  'review': "Prof. Thompson's passion for history is contagious. His assignments are challenging but fair."},
 {'professor': 'Dr. Lisa Patel',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Dr. Patel is an excellent teacher who goes above and beyond to ensure students grasp difficult concepts.'},
 {'professor': 'Prof.

In [4]:
import json
data = json.load(open("reviews.json"))
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )

    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review['review'],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [5]:
processed_data[0]

{'values': [0.0008141825,
  0.0029650347,
  0.068416975,
  0.0448762,
  0.005103065,
  0.006635267,
  0.017027313,
  0.041901547,
  -0.008379027,
  0.019886568,
  0.012770484,
  -0.0040388578,
  -0.00012841805,
  0.014206523,
  0.0014512643,
  0.036670268,
  -0.032054428,
  -0.013988553,
  0.027387304,
  0.065596186,
  0.04095274,
  -0.017399143,
  0.04610709,
  -0.020322507,
  -0.029438786,
  -0.037875514,
  -0.0067250193,
  0.014937364,
  0.03364433,
  0.0012437118,
  0.05790312,
  -0.0080007855,
  -0.019194191,
  -0.025861513,
  -0.053389855,
  0.032669872,
  0.009430413,
  -0.0010433716,
  0.04167076,
  0.013347464,
  0.011629347,
  -0.0041895136,
  -0.01223197,
  -0.005288981,
  0.04046551,
  -0.0026653258,
  0.008981651,
  -0.021117458,
  0.05523619,
  0.043901745,
  -0.033439178,
  0.010674125,
  0.023104832,
  -0.036901057,
  -0.025066564,
  -0.02338691,
  0.0034618783,
  0.022053447,
  0.014027018,
  -0.015770778,
  0.050569065,
  -0.007295588,
  -0.006564747,
  -0.017052956,


In [8]:
import base64

# Assuming your vectors are in a list of dictionaries
processed_data = [
    {
        "id": base64.urlsafe_b64encode(vec["id"].encode('utf-8')).decode('ascii'),
        "values": vec["values"],
        "metadata": vec["metadata"]
    }
    for vec in processed_data
]

index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")
# index = pc.Index("rag")
# upsert_response = index.upsert(
#     vectors=processed_data,
#     namespace="ns1",
# )
# print(f"Upserted count: {upsert_response['upserted_count']}")

Upserted count: 20


In [9]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}